# Обучение Word2Vec

In [88]:
#! pip install pymorphy2[fast] 
#! pip install natasha

In [90]:
import pandas as pd
import pymorphy2
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import json

from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    LOC,
    NamesExtractor,
    DatesExtractor,
    MoneyExtractor,
    AddrExtractor,
 
    Doc
)

In [4]:

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
def preprocess(text, stop_words, punctuation_marks, morph):
    tokens = word_tokenize(text.lower())
    preprocessed_text = []
    for token in tokens:
        if token not in punctuation_marks:
            lemma = morph.parse(token)[0].normal_form
            if lemma not in stop_words:
                preprocessed_text.append(lemma)
    return preprocessed_text

In [203]:
punctuation_marks = ['!', ',', '(', ')','[',']','',"'","''",',' ':', '-', '?', '..', '...','','"',"',",'`','``','"""',"''",'i','ii','iii']
stop_words = stopwords.words("russian")
morph = pymorphy2.MorphAnalyzer()

In [23]:
filename = '/content/merged_data.json'  #json lines file


In [137]:
segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)
dates_extractor = DatesExtractor(morph_vocab)
money_extractor = MoneyExtractor(morph_vocab)
addr_extractor = AddrExtractor(morph_vocab)

In [146]:
doc = Doc(FIO[0])


In [147]:
doc = Doc(FIO[0])
doc.segment(segmenter)
doc.tag_morph(morph_tagger)
display(doc.tokens)

doc.segment(segmenter) 
doc.tag_morph(morph_tagger)
doc.parse_syntax(syntax_parser)
doc.tag_ner(ner_tagger)

[DocToken(stop=15, text='Ответственность', pos='NOUN', feats=<Inan,Acc,Fem,Sing>),
 DocToken(start=16, stop=18, text='за', pos='ADP'),
 DocToken(start=19, stop=29, text='исполнение', pos='NOUN', feats=<Inan,Acc,Neut,Sing>),
 DocToken(start=30, stop=42, text='распоряжения', pos='NOUN', feats=<Inan,Gen,Neut,Sing>),
 DocToken(start=43, stop=52, text='возложить', pos='VERB', feats=<Perf,Inf,Act>),
 DocToken(start=53, stop=55, text='на', pos='ADP'),
 DocToken(start=56, stop=67, text='специалиста', pos='NOUN', feats=<Anim,Acc,Masc,Sing>),
 DocToken(start=68, stop=74, text='первой', pos='ADJ', feats=<Gen,Pos,Fem,Sing>),
 DocToken(start=75, stop=84, text='категории', pos='NOUN', feats=<Inan,Gen,Fem,Sing>),
 DocToken(start=85, stop=115, text='административно-хозяйственного', pos='ADJ', feats=<Gen,Pos,Masc,Sing>),
 DocToken(start=116, stop=122, text='отдела', pos='NOUN', feats=<Inan,Gen,Masc,Sing>),
 DocToken(start=123, stop=131, text='Кравцову', pos='PROPN', feats=<Anim,Dat,Masc,Sing>),
 DocTok

In [158]:
words = []
for tocken in doc.tokens:
    words.append(tocken.text)
  
doc1 = Doc(str(words))

Леммы

In [159]:
doc1.segment(segmenter) 
doc1.tag_morph(morph_tagger)
doc1.parse_syntax(syntax_parser)
doc1.tag_ner(ner_tagger)

for token in doc1.tokens:
    token.lemmatize(morph_vocab)
    
{_.text: _.lemma for _ in doc1.tokens}


{'[': '[',
 "'": "'",
 'Ответственность': 'ответственность',
 ',': ',',
 'за': 'за',
 'исполнение': 'исполнение',
 'распоряжения': 'распоряжение',
 'возложить': 'возложить',
 'на': 'на',
 'специалиста': 'специалист',
 'первой': 'первый',
 'категории': 'категория',
 'административно-хозяйственного': 'административно-хозяйственный',
 'отдела': 'отдел',
 'Кравцову': 'кравцов',
 'И': 'и',
 '.': '.',
 'Е': 'е',
 ']': ']'}

In [168]:
words_inf_form = {_.text: _.lemma for _ in doc1.tokens}.values()

In [204]:
new_words = []

for word in words_inf_form:
  for p in punctuation_marks:
    if word == p:
      word = word.replace(p,'')
  new_words.append(word)
      
        

In [205]:
new_words

['',
 '',
 'ответственность',
 '',
 'за',
 'исполнение',
 'распоряжение',
 'возложить',
 'на',
 'специалист',
 'первый',
 'категория',
 'административно-хозяйственный',
 'отдел',
 'кравцов',
 'и',
 '.',
 'е',
 '']

ФИО


In [120]:
for span in doc.spans:
    span.normalize(morph_vocab)

{_.text: _.normal for _ in doc.spans if _.text != _.normal}

{'Кравцову И.Е.': 'Кравцов И.Е.'}

In [25]:
import pandas as pd
df = pd.read_json(filename)
df.head()

df = pd.read_json(filename, lines=True)

In [34]:
df[0][0]['Tasks']


{'Task1': {'task_text': 'Закупить книги. Срок завершения: 19 апр 15.',
  'task_responsibles_people': 'Ответственность за исполнение распоряжения возложить на специалиста первой категории административно-хозяйственного отдела Кравцову И.Е.',
  'task_responsibles_groups': 'Административно-хозяйственный отдел',
  'task_deadline': ['19 апр 15', 1429383600.0]},
 'Global_supervisor': ['Ответственность за исполнение распоряжения возложить на специалиста первой категории административно-хозяйственного отдела Кравцову И.Е.'],
 'Global_deadline': ['11 апреля 2013 года', 1365616800.0]}

In [38]:
departments = []
for n in range(10000):
    for i in range(10):
      try:
        departments.append(df[n][i]['Tasks']['Task1']['task_responsibles_groups'])
      except KeyError:
        continue
departments

['Административно-хозяйственный отдел',
 'Служба ИТ',
 'Отдел разработки систем',
 'Бухгалтерия',
 'Бухгалтерия',
 'Отдел проектирования',
 'Бухгалтерия',
 'Отдел разработки систем',
 'Отдел разработки систем',
 'Служба ИТ',
 'Отдел проектирования',
 'Административно-хозяйственный отдел',
 'Отдел архитектуры',
 'Отдел разработки систем',
 'Административно-хозяйственный отдел',
 'Бухгалтерия',
 'Отдел разработки систем',
 'Бухгалтерия',
 'Отдел проектирования',
 'Отдел проектирования',
 'Административно-хозяйственный отдел',
 'Отдел разработки систем',
 'Административно-хозяйственный отдел',
 'Бухгалтерия',
 'Отдел эксплуатации',
 'Отдел разработки систем',
 'Служба ИТ',
 'Служба ИТ',
 'Отдел разработки систем',
 'Отдел эксплуатации',
 'Служба ИТ',
 'Отдел проектирования',
 'Бухгалтерия',
 'Отдел разработки систем',
 'Административно-хозяйственный отдел',
 'Бухгалтерия',
 'Отдел архитектуры',
 'Служба ИТ',
 'Бухгалтерия',
 'Административно-хозяйственный отдел',
 'Служба ИТ',
 'Служба ИТ

In [107]:
FIO = []
for n in range(10000):
  for i in range(10):
      try:
        FIO.append(df[n][i]['Tasks']['Task1']['task_responsibles_people'])
      except KeyError:
        continue


In [106]:
POSITION = []
for n in range(10000):
    try:
        for i in range(10):
            POSITION.append(df[n]['Tasks']['Global_supervisor'])
    except KeyError:
            continue

## Обучение Word2Vec на отзывах на банки

In [52]:
import gensim.models

In [207]:
morph = pymorphy2.MorphAnalyzer()

In [229]:
vocab, index = {}, 1  # start indexing from 1
vocab['<pad>'] = 0  # add a padding token
for token in new_words:
  if token not in vocab:
    vocab[token] = index
    index += 1
vocab_size = len(vocab)
print(vocab)


{'<pad>': 0, '': 1, 'ответственность': 2, 'за': 3, 'исполнение': 4, 'распоряжение': 5, 'возложить': 6, 'на': 7, 'специалист': 8, 'первый': 9, 'категория': 10, 'административно-хозяйственный': 11, 'отдел': 12, 'кравцов': 13, 'и': 14, '.': 15, 'е': 16}


In [288]:
# исправить df['ключи'] на список или слова

model = gensim.models.Word2Vec(sentences=list(df['ключи']), 
                               min_count=1, 
                               size=52)

In [287]:
model.wv['т']


array([ 0.0070841 , -0.00100014, -0.009538  ,  0.00188037, -0.00471181,
        0.00788026, -0.00331786, -0.00419109,  0.00810119,  0.00350038,
        0.00565284, -0.00679533,  0.00710863,  0.00564418,  0.00371613,
        0.00709172, -0.00053789, -0.00361284, -0.00273121,  0.00263621,
       -0.00744932,  0.00231424, -0.00805624, -0.00065361, -0.00199067,
        0.00535866,  0.00890043, -0.00148795,  0.00058114, -0.00435064,
       -0.00153049,  0.00551062,  0.0002367 , -0.00218917,  0.00730623,
        0.00261322,  0.0031825 , -0.00152465,  0.00072524,  0.00604659,
       -0.00657239,  0.00174165, -0.00164758, -0.00014246,  0.00129589,
        0.00553248, -0.00827978, -0.00523773,  0.00393518,  0.0051037 ,
       -0.00358029,  0.00335394], dtype=float32)